In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, translate,when,lit
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.ml.feature import Imputer

In [2]:
spark = (SparkSession
.builder
.appName("Spark_ML_APP")
.enableHiveSupport() # we need this to create tables
.getOrCreate())
sc = spark.sparkContext
sc.setLogLevel("WARN")

22/07/14 16:09:48 WARN Utils: Your hostname, ivo-Nitro-AN515-57 resolves to a loopback address: 127.0.1.1; using 192.168.1.197 instead (on interface wlp0s20f3)
22/07/14 16:09:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/14 16:09:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
filepath ="""../data/sf-airbnb.csv"""

In [4]:
# inferring schema from a small sample
sample_df = spark.read.option("samplingRation",0.001)\
            .option("header",True).option("multiline",True).csv(filepath)
sample_df.printSchema()
airbnb_schema = sample_df.schema
print(len(airbnb_schema.fieldNames())) # -> 106


root
 |-- id: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: string (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- space: string (nullable = true)
 |-- description: string (nullable = true)
 |-- experiences_offered: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- transit: string (nullable = true)
 |-- access: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- house_rules: string (nullable = true)
 |-- thumbnail_url: string (nullable = true)
 |-- medium_url: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- xl_picture_url: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_location: string (nullable

In [5]:
sample_df.select('amenities').show(n=1, truncate=False)
# we have double quotes at the begining, and we're losing data... 

+-----------------+
|amenities        |
+-----------------+
|"{TV,""Cable TV""|
+-----------------+
only showing top 1 row



In [6]:
# get the sf-fire data from reading directly the file
airbnb_df = spark.read.csv(filepath, header = True, schema= airbnb_schema,multiLine=True,escape='"')
# Multiline -> parse one record, which may span multiple lines, per file. CSV built-in functions ignore this option.
# escape -> will remove double quotes at the begining

In [7]:
airbnb_df.select('amenities').show(n=1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|amenities                                                                                                                                                                                                                                                                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
# we have many columns, so we'll just keep this in line with the book
columnsToKeep = [
  "host_is_superhost",
  "cancellation_policy",
  "instant_bookable",
  "host_total_listings_count",
  "neighbourhood_cleansed",
  "latitude",
  "longitude",
  "property_type",
  "room_type",
  "accommodates",
  "bathrooms",
  "bedrooms",
  "beds",
  "bed_type",
  "minimum_nights",
  "number_of_reviews",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value",
  "price"]

In [9]:
airbnb_df = airbnb_df.select(columnsToKeep)
airbnb_df.cache().count()
display(airbnb_df)

DataFrame[host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: string, neighbourhood_cleansed: string, latitude: string, longitude: string, property_type: string, room_type: string, accommodates: string, bathrooms: string, bedrooms: string, beds: string, bed_type: string, minimum_nights: string, number_of_reviews: string, review_scores_rating: string, review_scores_accuracy: string, review_scores_cleanliness: string, review_scores_checkin: string, review_scores_communication: string, review_scores_location: string, review_scores_value: string, price: string]

We have several quantitive categories as string. E.g.  host_total_listings_count, bedrooms, price, etc.

In [10]:
airbnb_df.select("host_total_listings_count").collect()[0]

Row(host_total_listings_count='1')

In [11]:
airbnb_df.select("price").collect()[0]

Row(price='$170.00')

In [12]:
airbnb_df = airbnb_df.withColumn("host_total_listings_count", col("host_total_listings_count").cast("int"))
airbnb_df = airbnb_df.withColumn("bathrooms", col("bathrooms").cast("int"))
airbnb_df = airbnb_df.withColumn("bedrooms", col("bedrooms").cast("int"))
airbnb_df = airbnb_df.withColumn("price", translate(col("price"), "$,", "").cast("double"))
# we could clean more, but this is just a p-o-c.
display(airbnb_df)


DataFrame[host_is_superhost: string, cancellation_policy: string, instant_bookable: string, host_total_listings_count: int, neighbourhood_cleansed: string, latitude: string, longitude: string, property_type: string, room_type: string, accommodates: string, bathrooms: int, bedrooms: int, beds: string, bed_type: string, minimum_nights: string, number_of_reviews: string, review_scores_rating: string, review_scores_accuracy: string, review_scores_cleanliness: string, review_scores_checkin: string, review_scores_communication: string, review_scores_location: string, review_scores_value: string, price: double]

In [13]:
airbnb_df.select("price").summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              7151|
|   mean| 213.6540344007831|
| stddev|313.28222046853125|
|    min|               0.0|
|    25%|             100.0|
|    50%|             150.0|
|    75%|             235.0|
|    max|           10000.0|
+-------+------------------+



In [14]:
airbnb_df.summary().show(vertical=True)
# press the rightmost blue link below to see the full results.

-RECORD 0-------------------------------------------
 summary                     | count                
 host_is_superhost           | 7151                 
 cancellation_policy         | 7151                 
 instant_bookable            | 7151                 
 host_total_listings_count   | 7151                 
 neighbourhood_cleansed      | 7151                 
 latitude                    | 7151                 
 longitude                   | 7151                 
 property_type               | 7151                 
 room_type                   | 7151                 
 accommodates                | 7151                 
 bathrooms                   | 7130                 
 bedrooms                    | 7149                 
 beds                        | 7144                 
 bed_type                    | 7151                 
 minimum_nights              | 7151                 
 number_of_reviews           | 7151                 
 review_scores_rating        | 5730           

### Nulls and Imputation


If we do any imputation techniques for categorical/numerical features, 
we MUST include an additional column specifying that field was imputed.

In [15]:
cols_with_nulls = [x for x in airbnb_df.columns if airbnb_df.filter(col(x).isNull()).count() > 0]
cols_with_nulls # columns with null values

['bathrooms',
 'bedrooms',
 'beds',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

In [16]:
air_test= airbnb_df

In [17]:
air_test=(air_test.withColumn("bathrooms_null_dummy", 
                    col("bathrooms").isNull().cast("int")))

In [18]:
air_test.filter(col("bathrooms").isNull()).select("bathrooms").count()

21

In [19]:
air_test.filter(col("bathrooms_null_dummy") == 1).select("bathrooms").count()
# we've created a one-hot encoder for Null values in bathrooms column

21

In [20]:
airbnb_df = (airbnb_df.withColumn("bathrooms_null_dummy", 
                    col("bathrooms").isNull().cast("int")))

In [21]:
airbnb_df = airbnb_df.na.drop(subset=["host_is_superhost"])

Using SparkML's Imputer

In [19]:
imputeCols = cols_with_nulls

#  [
#   "bedrooms",
#   "bathrooms",
#   "beds", 
#   "review_scores_rating",
#   "review_scores_accuracy",
#   "review_scores_cleanliness",
#   "review_scores_checkin",
#   "review_scores_communication",
#   "review_scores_location",
#   "review_scores_value"
# ]
imputeCols

['bathrooms',
 'bedrooms',
 'beds',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

In [20]:
# we need to cast everything to double
integerColumns = [x.name for x in airbnb_df.schema.fields 
                   if x.dataType != DoubleType() and x.name in imputeCols]

for c in integerColumns:
  airbnb_df = airbnb_df.withColumn(c, col(c).cast("double"))

columns = "\n - ".join(integerColumns)
print(f"Columns converted from Integer to Double (in imputeCols):\n - {columns}")

Columns converted from Integer to Double (in imputeCols):
 - bathrooms
 - bedrooms
 - beds
 - review_scores_rating
 - review_scores_accuracy
 - review_scores_cleanliness
 - review_scores_checkin
 - review_scores_communication
 - review_scores_location
 - review_scores_value


In [21]:
for c in imputeCols:
  airbnb_df = airbnb_df.withColumn(c + "_na_dummy", when(col(c).isNull(), 1.0).otherwise(0.0))

In [179]:
airbnb_df.columns

['host_is_superhost',
 'cancellation_policy',
 'instant_bookable',
 'host_total_listings_count',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'minimum_nights',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'price',
 'bedrooms_na_dummy',
 'bathrooms_na_dummy',
 'beds_na_dummy',
 'review_scores_rating_na_dummy',
 'review_scores_accuracy_na_dummy',
 'review_scores_cleanliness_na_dummy',
 'review_scores_checkin_na_dummy',
 'review_scores_communication_na_dummy',
 'review_scores_location_na_dummy',
 'review_scores_value_na_dummy']

In [22]:

imputer = Imputer(strategy="median", inputCols=imputeCols, outputCols=imputeCols)
# we need to define the outputCols as the imputeCols when doing multicolumn imputation
airbnb_df = imputer.fit(airbnb_df).transform(airbnb_df)

In [23]:
cols_with_nulls_2 = [x for x in airbnb_df.columns if airbnb_df.filter(col(x).isNull()).count() > 0]
cols_with_nulls_2 # empty because we've imputed the non-null values into the columns

[]